In [1]:
import SpaDecon as spd
import scanpy as sc
import pandas as pd
import numpy as np
from skimage import io
import timeit

Using TensorFlow backend.


In [2]:
#Please download the spadecon_tutorial_data folder from https://drive.google.com/drive/folders/1_eBGKMVYh4p1eiHhrJASnqOD9z_CP5e0?usp=sharing
#set working directory to spadecon_tutorial_data using os.chdir()
adata_sc = sc.read('bc_sc.h5ad')
adata_st = sc.read_10x_h5('V1_Breast_Cancer_Block_A_Section_1_filtered_feature_bc_matrix.h5')
adata_st.var_names_make_unique()
locations = pd.read_csv('tissue_positions_list.csv', header = None, index_col = 0)
locations = locations.loc[adata_st.obs.index]
histology = io.imread("V1_Breast_Cancer_Block_A_Section_1_image.tif")
np.random.seed(10)
clf = spd.SpaDecon()
clf.deconvolution(adata_sc, adata_st, histology, locations, technology = 'Visium')

OSError: Unable to open file (unable to open file: name = 'bc_sc.h5ad', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [3]:
spadecon_props = clf.props
spadecon_props.to_csv('spadecon_props.csv')

92.814018887